In [ ]:
#dev branch test
#include("fileio.jl")
#include("kabsch_umeyama.jl")
include("foldexity.jl")


In [ ]:
pdb = readpdb("testpdb/small.pdb")
writepdb(pdb, "testpdb/small_backbone.pdb")

xyzcoords = pdb2matrix(pdb)
coordmatrix = matrix2fragments(xyzcoords)

pdbmatrix = pdb2pdbmatrix(pdb)
pdbfragments = matrix2fragments(pdbmatrix)

println()


In [ ]:
pfrag1 = coordmatrix[10]
pfrag2 = coordmatrix[31]

align_rmsd(pfrag1, pfrag2)

In [ ]:
function kabsh_matrix(megax)    
    try        
        n = size(megax)[1]  # Change this to the desired size
        matrix = zeros(Float64, n, n)

        for i = 1:n # Fill the upper triangle
            for j = i+1:n  # Ensure j >= i for the upper triangle
                matrix[i, j] = align_rmsd(megax[i], megax[j])
            end
        end
        return sum(matrix) * 2 / n , matrix
        
    catch 
        return 0
    end
end

In [ ]:
fx, m = kabsh_matrix(coordmatrix)

In [ ]:
m[m .< 1] .=0
m[m .>= 1] .=1


In [80]:
sum(m)/size(m)[1]

11.638297872340425

In [ ]:
data = fxdir("testpdb/", true)  

In [ ]:
fxity, nfrags = fxpdb("testpdb/small.pdb")  